In [10]:
import json
import carla
import math
import random
import threading
from agents.navigation.global_route_planner import GlobalRoutePlanner
from agents.navigation.behavior_agent import BehaviorAgent

def read_json_scenarios(file_name):
    json_list = []  # Lista per accumulare i JSON
    try:
        with open(nome_file, "r", encoding="utf-8") as file:
            for i, r in enumerate(file, start=1):
                r = r.strip()
                if r: #check if trimmed row is not empty
                    try:
                        json_obj = json.loads(r)  #read row as json struct
                        json_list.append(json_obj)
                    except json.JSONDecodeError as e:
                        print(f"Error in row {i}: invalid JSON format ({e})")
        return json_list
    except FileNotFoundError:
        print(f"File '{nome_file}' missing!!")
    except Exception as e:
        print(f"Error during parsing file '{nome_file}': {e}")
    return []


def generate_scenarios_from_file(world, file_name):
    modes = ["cautious", "normal", "aggressive"]
    scenarios = []
    #iterating through file
    for j in read_json_scenarios(file_name):
        
        ego_spawn = j['ego_spawn']
        npc_spawn = j['npc_spawn']
        
        ego_dest = j['ego_dest']
        npc_dest = j['npc_dest']
        
        lead_transform = carla.Transform(carla.Location(x=ego_spawn['x'], y=ego_spawn['y'], z=ego_spawn['z']), carla.Rotation(pitch=ego_spawn['pitch'], yaw=ego_spawn['yaw'], roll=ego_spawn['roll']))
        lag_transform = carla.Transform(carla.Location(x=npc_spawn['x'], y=npc_spawn['y'], z=npc_spawn['z']), carla.Rotation(pitch=npc_spawn['pitch'], yaw=npc_spawn['yaw'], roll=npc_spawn['roll']))
        
        carla_map = world.get_map()
        dest_lag_wp = carla_map.get_waypoint( carla.Location(x=ego_dest['x'], y=ego_dest['y'], z=ego_dest['z']) )
        dest_lead_wp = carla_map.get_waypoint( carla.Location(x=npc_dest['x'], y=npc_dest['y'], z=npc_dest['z']) )
    
        #creating all scenarios
        for outer_mode in modes:
            for inner_mode in modes:
                scenarios.append( (lag_transform, dest_lag_wp, outer_mode, lead_transform, dest_lead_wp, inner_mode) )
    return scenarios

#MAIN

client = carla.Client('localhost', 2000)
client.set_timeout(5.0)
world = client.get_world()

scenarios = generate_scenarios_from_file(world, "sim_data_all.log")
for i in scenarios:
   print(i)
print(len(scenarios))

print("PROGRAM ENDED")


(<carla.libcarla.Transform object at 0x000001BFAD61EB60>, <carla.libcarla.Waypoint object at 0x000001BFAD5AE810>, 'cautious', <carla.libcarla.Transform object at 0x000001BFAD61ECF0>, <carla.libcarla.Waypoint object at 0x000001BFAD5ACE40>, 'cautious')
(<carla.libcarla.Transform object at 0x000001BFAD61EB60>, <carla.libcarla.Waypoint object at 0x000001BFAD5AE810>, 'cautious', <carla.libcarla.Transform object at 0x000001BFAD61ECF0>, <carla.libcarla.Waypoint object at 0x000001BFAD5ACE40>, 'normal')
(<carla.libcarla.Transform object at 0x000001BFAD61EB60>, <carla.libcarla.Waypoint object at 0x000001BFAD5AE810>, 'cautious', <carla.libcarla.Transform object at 0x000001BFAD61ECF0>, <carla.libcarla.Waypoint object at 0x000001BFAD5ACE40>, 'aggressive')
(<carla.libcarla.Transform object at 0x000001BFAD61EB60>, <carla.libcarla.Waypoint object at 0x000001BFAD5AE810>, 'normal', <carla.libcarla.Transform object at 0x000001BFAD61ECF0>, <carla.libcarla.Waypoint object at 0x000001BFAD5ACE40>, 'cautious'